In [ ]:
from keras.models import Sequential, Model
#from keras.applications import MobileNet
from keras.preprocessing.image import ImageDataGenerator 
from keras.preprocessing import image 
from keras.utils import to_categorical
from keras.applications.vgg19 import VGG19
from keras.layers import Dense, MaxPooling2D, Flatten, Activation, Dropout,GlobalAveragePooling2D
from keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
import os
from os import listdir
from os.path import isfile, join
import numpy as np
import cv2
import keras
import matplotlib.pyplot as plt 
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [ ]:
check_point  = ModelCheckpoint("face_emotion_mobilenet.h5",
                               monitor="val_loss",
                               mode = "min",
                               save_best_only=True,
                               verbose=1)
early_stop = EarlyStopping(monitor="val_loss",
                          min_delta=0,
                          patience=10,
                          verbose=1,restore_best_weights=True)
learning_rate_reduction = ReduceLROnPlateau(monitor="val_accuracy",
                                           patience=5,
                                           verbose=1,
                                           factor=0.2,
                                           min_lr=0.0001)
callbacks = [early_stop,check_point,learning_rate_reduction]

In [ ]:
num_class = 5
r,c = (224,224)
batch_size = 32
train_data_dir = 'train'
valid_data_dir = 'validation'

train_datagen = ImageDataGenerator(rescale=1./255,
                                  rotation_range=30,
                                  shear_range=0.3,
                                  zoom_range=0.3,
                                  width_shift_range=0.4,
                                  height_shift_range=0.4,
                                  horizontal_flip=True,
                                  fill_mode="nearest")

valid_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(train_data_dir,
                                              class_mode="categorical",
                                              target_size=(r,c),
                                              batch_size=batch_size)

valid_gen = valid_datagen.flow_from_directory(valid_data_dir,
                                              class_mode="categorical",
                                              target_size=(r,c),
                                              batch_size=batch_size)

In [ ]:
m = VGG16(weights='imagenet',include_top=False,input_shape=(r,c,3))
for layer in m.layers:
    layer.trainable = True 


In [ ]:
x = m.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation="relu")(x)
x = Dense(1024, activation="relu")(x)
x = Dense(512, activation="relu")(x)
x = Dense(5,activation="softmax")(x)

In [ ]:
model = Model(inputs=m.input, outputs=x)

In [ ]:
model.summary()

In [ ]:
model.compile(loss = "categorical_crossentropy",optimizer=keras.optimizers.Adam(lr=0.001),metrics=["accuracy"])
num_train = 24256
num_valid = 3006
epochs = 30
his = model.fit_generator(train_gen,
                     steps_per_epoch = num_train//batch_size,
                     epochs = epochs,callbacks=callbacks,
                     validation_data = valid_gen,
                     validation_steps = num_valid//batch_size)

In [ ]:
model.save("Mobile_Net.h5")

In [ ]:
plt.plot(his.history["accuracy"])
plt.plot(his.history["val_accuracy"])
plt.title("Model_Accuracy")
plt.ylabel("Accuracy")
plt.xlabel("epochs")
plt.legend(['Train',"Test"],loc="upper left")
plt.show()

In [ ]:
plt.plot(his.history["loss"])
plt.plot(his.history["val_loss"])
plt.title("Model_loss")
plt.ylabel("Loss")
plt.xlabel("epochs")
plt.legend(['Train',"Test"],loc="upper left")
plt.show()

In [ ]:
y_pred = model.predict_generator(valid_gen,num_valid//batch_size+1)
y_pred = np.argmax(y_pred,axis=1)
print("Confution matrix")
print(confusion_matrix(valid_gen.classes,y_pred))